# Command and Control Application Suite E2E Demonstration

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

This notebook demonstrates the current implementation of the command and control (C2) server and beacon applications in primAITE.

In [ ]:
# Imports
from primaite.config.load import data_manipulation_config_path
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.game.agent.interface import AgentHistoryItem
import yaml
from pprint import pprint
from primaite.simulator.network.container import Network
from primaite.game.game import PrimaiteGame
from primaite.simulator.system.applications.application import ApplicationOperatingState
from primaite.simulator.system.applications.red_applications.c2.c2_beacon import C2Beacon
from primaite.simulator.system.applications.red_applications.c2.c2_server import C2Server
from primaite.simulator.system.applications.red_applications.c2.abstract_c2 import C2Command, C2Payload
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript
from primaite.simulator.system.software import SoftwareHealthState
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server

## **Notebook Setup** | **Network Configuration:**

This notebook uses the same network setup as UC2. Please refer to the main [UC2-E2E-Demo notebook for further reference](./Data-Manipulation-E2E-Demonstration.ipynb).

However, this notebook will replaces with the red agent used in UC2 with a custom proxy red agent built for this notebook.

In [ ]:
custom_c2_agent = """
  - ref: CustomC2Agent
    team: RED
    type: ProxyAgent
    observation_space: null
    action_space:
      action_list:
        - type: DONOTHING
        - type: NODE_APPLICATION_INSTALL
        - type: NODE_APPLICATION_EXECUTE
        - type: CONFIGURE_C2_BEACON
        - type: C2_SERVER_RANSOMWARE_LAUNCH
        - type: C2_SERVER_RANSOMWARE_CONFIGURE
        - type: C2_SERVER_TERMINAL_COMMAND
      options:
        nodes:
          - node_name: web_server
            applications: 
              - application_name: C2Beacon
          - node_name: client_1
            applications: 
              - application_name: C2Server
        max_folders_per_node: 1
        max_files_per_folder: 1
        max_services_per_node: 2
        max_nics_per_node: 8
        max_acl_rules: 10
        ip_list:
          - 192.168.1.21
          - 192.168.1.14
        wildcard_list:
          - 0.0.0.1
      action_map:
        0:
          action: DONOTHING
          options: {}
        1:
          action: NODE_APPLICATION_INSTALL
          options:
            node_id: 0
            application_name: C2Beacon
        2:
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0
            config:
              c2_server_ip_address: 192.168.10.21
              keep_alive_frequency:
              masquerade_protocol:
              masquerade_port:
        3:
          action: NODE_APPLICATION_EXECUTE
          options:
            node_id: 0
            application_id: 0  
        4:
          action: C2_SERVER_TERMINAL_COMMAND
          options:
            node_id: 1
            ip_address:
            account:
              username: admin
              password: admin
            commands:
              - 
                - software_manager
                - application
                - install
                - RansomwareScript
        5:
          action: C2_SERVER_RANSOMWARE_CONFIGURE
          options:
            node_id: 1
            config:
              server_ip_address: 192.168.1.14
              payload: ENCRYPT
        6:
          action: C2_SERVER_RANSOMWARE_LAUNCH
          options:
            node_id: 1
        7:
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0
            config:
              c2_server_ip_address: 192.168.10.21
              keep_alive_frequency: 10
              masquerade_protocol: TCP
              masquerade_port: DNS



    reward_function:
      reward_components:
        - type: DUMMY
"""
c2_agent_yaml = yaml.safe_load(custom_c2_agent)

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = c2_agent_yaml
    

env = PrimaiteGymEnv(env_config=cfg)

## **Notebook Setup** | Network Prerequisites

Before the Red Agent is able to perform any C2 specific actions, the C2 Server needs to be installed and run before the episode begins.

This is because higher fidelity environments (and the real-world) a C2 server would not be accessible by private network blue agent and the C2 Server would already be in place before the an adversary (Red Agent) before the narrative of the use case.

The cells below installs and runs the C2 Server on the client_1 directly via the simulation API.

In [ ]:
client_1: Computer = env.game.simulation.network.get_node_by_hostname("client_1")
client_1.software_manager.install(C2Server)
c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server.run()
client_1.software_manager.show()

## **Command and Control** | C2 Beacon Actions

Before any C2 Server commands is able to accept any commands, it must first establish connection with a C2 beacon.

This can be done by installing, configuring and then executing a C2 Beacon. 

### **Command and Control** | C2 Beacon Actions | Installation

In [ ]:
env.step(1)
web_server: Computer = env.game.simulation.network.get_node_by_hostname("web_server")
web_server.software_manager.show()

### **Command and Control** | C2 Beacon Actions | Configuration

In [ ]:
env.step(2)
c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]
web_server.software_manager.show()
c2_beacon.show()

### **Command and Control** | C2 Beacon Actions | Establishing Connection

In [ ]:
env.step(3) 

In [ ]:
c2_beacon.show()
c2_server.show()

## **Command and Control** | C2 Server Actions

### **Command and Control** | C2 Server Actions | Executing Terminal Commands

In [ ]:
env.step(4)

In [ ]:
client_1.software_manager.show()

### **Command and Control** | C2 Server Actions | Configuring Ransomware

In [ ]:
env.step(5)

In [ ]:
env.step(6)

In [ ]:
ransomware_script: RansomwareScript = web_server.software_manager.software["RansomwareScript"]
web_server.software_manager.show()
ransomware_script.show()

### **Command and Control** | C2 Server Actions | Launching Ransomware

In [ ]:
env.step(6)

In [ ]:
database_server: Server = env.game.simulation.network.get_node_by_hostname("database_server")
database_server.software_manager.file_system.show(full=True)

## **Command and Control** | Blue Agent Relevance

The next section of the notebook will demonstrate the impact that the command and control suite has to the Blue Agent's observation space as well as some potential actions that can be used to prevent the attack from being successfully.

The code cell below re-creates the UC2 network and swaps out the previous custom red agent with a custom blue agent. 


In [ ]:
custom_blue_agent_yaml = """  
  - ref: defender
    team: BLUE
    type: ProxyAgent

    observation_space:
      type: CUSTOM
      options:
        components:
          - type: NODES
            label: NODES
            options:
              hosts:
                - hostname: web_server
                  applications:
                    - application_name: C2Beacon
                    - application_name: RansomwareScript
                - hostname: database_server
                  folders:
                    - folder_name: database
                      files:
                      - file_name: database.db
                - hostname: client_1
                - hostname: client_2
              num_services: 0
              num_applications: 2
              num_folders: 1
              num_files: 1
              num_nics: 0
              include_num_access: false
              include_nmne: false
              monitored_traffic:
                icmp:
                    - NONE
                tcp:
                    - HTTP
              routers:
                - hostname: router_1
              num_ports: 1
              ip_list:
                - 192.168.10.21
                - 192.168.1.12
              wildcard_list:
                - 0.0.0.1
              port_list:
                - 80
              protocol_list:
                - ICMP
                - TCP
                - UDP
              num_rules: 10

          - type: LINKS
            label: LINKS
            options:
              link_references:
                - router_1:eth-1<->switch_1:eth-8
                - router_1:eth-2<->switch_2:eth-8
                - switch_1:eth-1<->web_server:eth-1
                - switch_1:eth-2<->web_server:eth-1
                - switch_1:eth-3<->database_server:eth-1
                - switch_1:eth-4<->backup_server:eth-1
                - switch_1:eth-7<->security_suite:eth-1
                - switch_2:eth-1<->client_1:eth-1
                - switch_2:eth-2<->client_2:eth-1
                - switch_2:eth-7<->security_suite:eth-2
          - type: "NONE"
            label: ICS
            options: {}
    
    action_space:
      action_list:
        - type: NODE_APPLICATION_REMOVE
        - type: NODE_SHUTDOWN
        - type: ROUTER_ACL_ADDRULE
        - type: DONOTHING
      action_map:
          0:
            action: DONOTHING
            options: {}
          1:
            action: NODE_APPLICATION_REMOVE
            options:
              node_id: 0
              application_name: C2Beacon
          2:
            action: NODE_SHUTDOWN
            options:
              node_id: 0
          3:
            action: ROUTER_ACL_ADDRULE
            options:
              target_router: router_1
              position: 1
              permission: 2
              source_ip_id: 2
              dest_ip_id: 3
              source_port_id: 2
              dest_port_id: 2
              protocol_id: 1
              source_wildcard_id: 0
              dest_wildcard_id: 0  


      options:
        nodes:
        - node_name: web_server
          applications:
          - application_name: C2Beacon

        - node_name: database_server
          folders:
          - folder_name: database
            files:
            - file_name: database.db
          services:
          - service_name: DatabaseService
        - node_name: router_1

        max_folders_per_node: 2
        max_files_per_folder: 2
        max_services_per_node: 2
        max_nics_per_node: 8
        max_acl_rules: 10
        ip_list:
          - 192.168.10.21
          - 192.168.1.12
        wildcard_list:
          - 0.0.0.1
    reward_function:
      reward_components:
        - type: DUMMY

    agent_settings:
      flatten_obs: False
"""
custom_blue = yaml.safe_load(custom_blue_agent_yaml)

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = custom_blue
    

blue_env = PrimaiteGymEnv(env_config=cfg)

In [ ]:
# Utility function for showing OBS changes between each time step.

from deepdiff.diff import DeepDiff

def display_obs_diffs(old, new, step_counter):
    """
    Use DeepDiff to extract and display differences in old and new instances of
    the observation space.

    :param old: observation space instance.
    :param new: observation space instance.
    :param step_counter: current step counter.
    """
    print("\nObservation space differences")
    print("-----------------------------")
    diff = DeepDiff(old, new)
    print(f"Step {step_counter}")
    for d,v in diff.get('values_changed', {}).items():
        print(f"{d}: {v['old_value']} -> {v['new_value']}")

### **Command and Control** | Blue Agent Relevance | Observation Space

This section demonstrates the OBS impact if the C2 suite is successfully installed and then used to install, configure and launch the ransomwarescript.

In [ ]:
# Resetting the environment and capturing the default observation space.
blue_env.reset()
default_obs, _, _, _, _ = blue_env.step(0)

In [ ]:
# Setting up the C2 Suite via the simulation API.

client_1: Computer = blue_env.game.simulation.network.get_node_by_hostname("client_1")
web_server: Server = blue_env.game.simulation.network.get_node_by_hostname("web_server")

# Installing the C2 Server.
client_1.software_manager.install(C2Server)
c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server.run()

# Installing the C2 Beacon.
web_server.software_manager.install(C2Beacon)
c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]
c2_beacon.configure(c2_server_ip_address="192.168.10.21")
c2_beacon.establish()

In [ ]:
# Capturing the observation impacts of the previous code cell: C2 Suite setup.
c2_configuration_obs, _, _, _, _ = blue_env.step(0)

In [ ]:
display_obs_diffs(default_obs, c2_configuration_obs, blue_env.game.step_counter)

In [ ]:
# Installing RansomwareScript via C2 Terminal Commands
ransomware_install_command = {"commands":[["software_manager", "application", "install", "RansomwareScript"]],
                              "username": "pass123",
                              "password": "password123"}
c2_server._send_command(C2Command.TERMINAL, command_options=ransomware_install_command)


In [ ]:
# Configuring the RansomwareScript
ransomware_config = {"server_ip_address": "192.168.1.14", "payload": "ENCRYPT"}
c2_server._send_command(C2Command.RANSOMWARE_CONFIGURE, command_options=ransomware_config)

In [ ]:
# Capturing the observation impacts of the previous code cell: Ransomware installation & configuration.
c2_ransomware_obs, _, _, _, _ = blue_env.step(0)

The code cell below demonstrates the differences between the default observation space and the configuration of the C2 Server and the Ransomware installation.

In [ ]:
display_obs_diffs(default_obs, c2_ransomware_obs, env.game.step_counter)

In [ ]:
# Waiting for the ransomware to finish installing and then launching the RansomwareScript.
blue_env.step(0)
c2_server._send_command(C2Command.RANSOMWARE_LAUNCH, command_options={})

In [ ]:
# Capturing the observation impacts of the previous code cell: Launching the RansomwareScript.
c2_final_obs, _, _, _, _ = blue_env.step(0)

The code cell below demonstrates the differences between the default observation space and the configuration of the C2 Server, the ransomware script installation as well as the impact of RansomwareScript upon the database.

In [ ]:
display_obs_diffs(c2_ransomware_obs, c2_final_obs, blue_env.game.step_counter)

### **Command and Control** | Blue Agent Relevance | Action Space

The next section of this notebook will go over some potential blue agent actions that could be use to thwart the previously demonstrated attack.

In [ ]:
# This method is used to shorthand setting up the C2Server and the C2 Beacon.
def c2_setup(given_env: PrimaiteGymEnv):
    client_1: Computer = given_env.game.simulation.network.get_node_by_hostname("client_1")
    web_server: Server = given_env.game.simulation.network.get_node_by_hostname("web_server")

    client_1.software_manager.install(C2Server)
    c2_server: C2Server = client_1.software_manager.software["C2Server"]
    c2_server.run()

    web_server.software_manager.install(C2Beacon)
    c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]
    c2_beacon.configure(c2_server_ip_address="192.168.10.21")
    c2_beacon.establish()

    return given_env, c2_server, c2_beacon, client_1, web_server

#### Removing the C2 Beacon.

The simplest way a blue agent could prevent the C2 suite is by simply removing the C2 beacon from it's installation point. 

In [ ]:
blue_env.reset()

In [ ]:
# Setting up the C2 Suite using the c2_setup method & capturing the OBS impacts

blue_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_env)
pre_blue_action_obs, _, _, _, _ = blue_env.step(0)

The code cell below uses the custom blue agent defined at the start of this section perform NODE_APPLICATION_REMOVE on the C2 beacon

In [ ]:
# Using CAOS ACTION: NODE_APPLICATION_REMOVE & capturing the OBS
post_blue_action_obs, _, _, _, _ = blue_env.step(1)

Which we can see after the effects of after stepping another timestep and looking at the web_servers software manager and the OBS differences.

In [ ]:
blue_env.step(0)
web_server.software_manager.show()

In [ ]:
display_obs_diffs(pre_blue_action_obs, post_blue_action_obs, blue_env.game.step_counter)

Now we are unable to do so as the C2 Server is unable has lost it's connection to the C2 Beacon:

In [ ]:
# Attempting to install the C2 RansomwareScript
ransomware_install_command = {"commands":[["software_manager", "application", "install", "RansomwareScript"]],
                            "username": "pass123",
                            "password": "password123"}

c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server._send_command(C2Command.TERMINAL, command_options=ransomware_install_command)

#### Shutting down the node infected with a C2 Beacon.

Another way a blue agent can prevent the C2 suite is via shutting down the C2 beacon's host node. Whilst not as effective as the previous option, dependant on situation (such as multiple malicious applications) or other scenarios it may be more timestep efficient for a blue agent to shut down a node directly.

In [ ]:
blue_env.reset()

In [ ]:
# Setting up the C2 Suite using the c2_setup method & capturing the OBS impacts

blue_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_env)
pre_blue_action_obs, _, _, _, _ = blue_env.step(0)

The code cell below uses the custom blue agent defined at the start of this section perform NODE_SHUT_DOWN on the web server.

In [ ]:
# Using CAOS ACTION: NODE_SHUT_DOWN & capturing the OBS
post_blue_action_obs, _, _, _, _ = blue_env.step(2)

Which we can see after the effects of after stepping another timestep and looking at the web_servers operating state & the OBS differences.

In [ ]:
web_server = blue_env.game.simulation.network.get_node_by_hostname("web_server")
print(web_server.operating_state)

In [ ]:
display_obs_diffs(pre_blue_action_obs, post_blue_action_obs, blue_env.game.step_counter)

In [ ]:
# Attempting to install the C2 RansomwareScript
ransomware_install_command = {"commands":[["software_manager", "application", "install", "RansomwareScript"]],
                            "username": "pass123",
                            "password": "password123"}

c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server._send_command(C2Command.TERMINAL, command_options=ransomware_install_command)

#### Blocking C2 Traffic via ACL.

Another potential option a blue agent could take is by placing an ACL rule which blocks traffic between the C2 Server can C2 Beacon.

It's worth noting the potential effectiveness of approach is also linked by the current green agent traffic on the network. The same applies for the previous example.

In [ ]:
blue_env.reset()

In [ ]:
# Setting up the C2 Suite using the c2_setup method & capturing the OBS impacts

blue_env, c2_server, c2_beacon, client_1, web_server = c2_setup(given_env=blue_env)
pre_blue_action_obs, _, _, _, _ = blue_env.step(0)

The code cell below uses the custom blue agent defined at the start of this section to perform a ROUTER_ACL_ADDRULE on router 1.

In [ ]:
# Using CAOS ACTION: ROUTER_ACL_ADDRULE & capturing the OBS
post_blue_action_obs, _, _, _, _ = blue_env.step(3)

Which we can see after the effects of after stepping another timestep and looking at router 1's ACLs and the OBS differences.

In [ ]:
router_1: Router = blue_env.game.simulation.network.get_node_by_hostname("router_1")
router_1.acl.show()

Now we can see that the C2 applications are unable to maintain connection - thus being unable to execute correctly.

In [ ]:
blue_env.step(0)

# Attempting to install and execute the ransomware script
c2_server._send_command(C2Command.TERMINAL, command_options=ransomware_install_command)
c2_server._send_command(C2Command.RANSOMWARE_LAUNCH, command_options={})

In [ ]:
router_1.acl.show()

Because of the ACL rule the C2 beacon never received the ransomware installation and execute commands from the C2 server:

In [ ]:
web_server.software_manager.show()

In [ ]:
database_server: Server = blue_env.game.simulation.network.get_node_by_hostname("database_server")
database_server.software_manager.file_system.show(full=True)

In [ ]:
display_obs_diffs(pre_blue_action_obs, post_blue_action_obs, blue_env.game.step_counter)

## **Command and Control** | C2 Beacon Actions

Before any C2 Server commands is able to accept any commands, it must first establish connection with a C2 beacon.

This can be done by installing, configuring and then executing a C2 Beacon. 

## **Command and Control** | Configurability 

TODO: Fleshout

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = c2_agent_yaml
    

c2_config_env = PrimaiteGymEnv(env_config=cfg)

Installing the C2 Server

In [ ]:
client_1: Computer = c2_config_env.game.simulation.network.get_node_by_hostname("client_1")
client_1.software_manager.install(C2Server)
c2_server: C2Server = client_1.software_manager.software["C2Server"]
c2_server.run()
client_1.software_manager.show()

Installing the C2 Beacon via NODE_APPLICATION_INSTALL

In [ ]:
c2_config_env.step(1)

Configuring the C2 Beacon using different parameters:

``` yaml
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0
            config:
              c2_server_ip_address: 192.168.10.21
              keep_alive_frequency: 10
              masquerade_protocol: TCP
              masquerade_port: DNS
```

In [ ]:
c2_config_env.step(7)

In [ ]:
# Establishing connection to the C2 Server.
c2_config_env.step(3)

In [ ]:
web_server: Server = c2_config_env.game.simulation.network.get_node_by_hostname("web_server")
c2_beacon: C2Beacon = web_server.software_manager.software["C2Beacon"]
c2_beacon.show()
c2_server.show()